In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import IPython.display
import os
import nltk
import pickle
import re
from collections import Counter

In [2]:
data = pd.read_pickle("data/data_tokenized_sw_free_v2.pkl")
data.head()

,original_title,original_body,post,tags
Id,,,,
65526399,Groovy script code to append xml node taking 1...,<p>Below code is working but taking 15+ hours ...,node taking hour employee recordsgroovy xmlbe...,[javascript]
65526401,How do I make a class instance using user input?,<p>I am making a text based adventure game in ...,python usingi based ge ge begin create called...,[python]
65526407,"How to remove Branding WHMCS Ver 8.1 ""Powered ...",<p>I am a newbie and this is the first time ev...,remove ver powered whmcomplete solution whmcsi...,"[php, css]"
65526419,How can I construct my objects allocated throu...,<p>C++20 removed the <code>construct()</code> ...,object allocated allocate constructremoved des...,[c++]
65526447,React Router v5.2 - Blocking route change with...,<p>My app has two pages: <code>Step1</code> an...,react route change createbrowserhistory blockr...,"[javascript, reactjs]"


In [4]:
data.shape

(296944, 4)

# Partie supervisée

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, 
                                                    data.tags.to_list(),
                                                    test_size=0.2, 
                                                    random_state=42)

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
mlb.fit_transform(data.tags.to_list())
pickle.dump(mlb, open("data/binarizer.pkl", "wb"))

In [6]:
mlb = pickle.load(open("data/binarizer.pkl", "rb"))
mlb.classes_

array(['amazon-web-services', 'android', 'angular', 'c#', 'c++', 'css',
       'flutter', 'html', 'ios', 'java', 'javascript', 'node.js',
       'pandas', 'php', 'python', 'python-3.x', 'r', 'reactjs', 'swift',
       'typescript'], dtype=object)

In [7]:
from nltk import FreqDist

df = X_train.copy()
df.post = df.post.str.split()
vocabulary = set() 

for tag in mlb.classes_:
    df_tmp = df.copy()
    df_tmp = df_tmp[df_tmp.tags.map(lambda x: tag in x)]
    words = [w for liste in df_tmp.post.to_list() for w in liste]
    top100 = FreqDist(words).most_common(100)
    top100 = {x[0] for x in top100}
    vocabulary.update(top100)
    
len(vocabulary)

559

In [7]:
vocabulary = pickle.load(open("data/list_vocab.pkl", "rb"))

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

def splitter(x):
    return x.split()

vectorizer = TfidfVectorizer(vocabulary = vocabulary,
                             tokenizer = splitter)
pickle.dump(vectorizer, open("data/vectorizer.pkl", "wb"))

In [9]:
len(vectorizer.vocabulary)

559

In [10]:
X = vectorizer.fit_transform(X_train.post.to_list())
X_train_he = pd.DataFrame(data = X.toarray(),
                       columns = vectorizer.vocabulary)
X_train_he.head()

,change,fa,space,symfony,dplyr,found,swiftui,debug,studio,current,...,running,kafka,io,modal,score,formcontrol,cxx,axis,iloc,get
0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.00000,0.0,0.0,0.111293,0.0,0.0,0.0,0.0,...,0.0,0.0,0.107552,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.258368,0.0,0.00000,0.0,0.0,0.176705,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.364157,0.0,0.06152,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
X = vectorizer.fit_transform(X_test.post.to_list())
X_test_he = pd.DataFrame(data = X.toarray(),
                       columns = vectorizer.vocabulary)
X_test_he.head()

,change,fa,space,symfony,dplyr,found,swiftui,debug,studio,current,...,running,kafka,io,modal,score,formcontrol,cxx,axis,iloc,get
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.125428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.276235,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
train_labels = mlb.transform(y_train)
test_labels = mlb.transform(y_test)

In [13]:
dico_classes = dict(zip([i for i in range(len(mlb.classes_))], mlb.classes_))
dico_classes

{0: 'amazon-web-services',
 1: 'android',
 2: 'angular',
 3: 'c#',
 4: 'c++',
 5: 'css',
 6: 'flutter',
 7: 'html',
 8: 'ios',
 9: 'java',
 10: 'javascript',
 11: 'node.js',
 12: 'pandas',
 13: 'php',
 14: 'python',
 15: 'python-3.x',
 16: 'r',
 17: 'reactjs',
 18: 'swift',
 19: 'typescript'}

In [11]:
from sklearn.metrics import accuracy_score, average_precision_score, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier

In [32]:
from sklearn.svm import LinearSVC

svm_clf = OneVsRestClassifier(LinearSVC(dual = False))
svm_clf.fit(X_train_he, train_labels)
svm_preds = svm_clf.predict(X_test_he)

print('SVM precision score : ', precision_score(test_labels, svm_preds, average = 'samples'))
print('SVM recall score : ', recall_score(test_labels, svm_preds, average = 'samples'))
print('SVM accuracy score : ', accuracy_score(test_labels, svm_preds))

SVM precision score :  0.6578280489652967
SVM recall score :  0.6233455129176558
SVM accuracy score :  0.5089326306218324


/data/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
from sklearn.ensemble import RandomForestClassifier

rfc = OneVsRestClassifier(RandomForestClassifier(n_estimators=20))
rfc.fit(X_train_he, train_labels)
rfc_preds = rfc.predict(X_test_he)

print('rfc precision score : ', precision_score(test_labels, rfc_preds, average = 'samples'))
print('rfc recall score : ', recall_score(test_labels, rfc_preds, average = 'samples'))
print('rfc accuracy score : ', accuracy_score(test_labels, rfc_preds))

rfc precision score :  0.706575852991856
rfc recall score :  0.6659939831730904
rfc accuracy score :  0.5479634275707622


/data/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

ann = Sequential([
    layers.InputLayer(input_shape = X_train_he.shape[1]),
    layers.Dense(100, activation = "relu"),
    layers.Dense(100, activation = "relu"),
    layers.Dense(20, activation = "softmax")
])

ann.compile(optimizer = "adam",
            loss = "binary_crossentropy",
            metrics = ["accuracy"])

ann.fit(X_train_he, train_labels, epochs = 20, use_multiprocessing = True)
ann.save('data/ann.h5')

Epoch 1/20
7424/7424 [==============================] - 9s 1ms/step - loss: 0.0938 - accuracy: 0.6623
Epoch 2/20
7424/7424 [==============================] - 8s 1ms/step - loss: 0.0765 - accuracy: 0.7055
Epoch 3/20
7424/7424 [==============================] - 8s 1ms/step - loss: 0.0731 - accuracy: 0.7145
Epoch 4/20
7424/7424 [==============================] - 8s 1ms/step - loss: 0.0709 - accuracy: 0.7201
Epoch 5/20
7424/7424 [==============================] - 8s 1ms/step - loss: 0.0692 - accuracy: 0.7248
Epoch 6/20
7424/7424 [==============================] - 8s 1ms/step - loss: 0.0679 - accuracy: 0.7291
Epoch 7/20
7424/7424 [==============================] - 8s 1ms/step - loss: 0.0668 - accuracy: 0.7318
Epoch 8/20
7424/7424 [==============================] - 8s 1ms/step - loss: 0.0660 - accuracy: 0.7337
Epoch 9/20
7424/7424 [==============================] - 8s 1ms/step - loss: 0.0652 - accuracy: 0.7363
Epoch 10/20
7424/7424 [==============================] - 8s 1ms/step - loss: 0.064

In [125]:
ann.evaluate(X_test_he, test_labels)
# ann.history

1856/1856 [==============================] - 1s 670us/step - loss: 0.0770 - accuracy: 0.7130


[0.07699453830718994, 0.7130445241928101]

In [14]:
from tensorflow.keras.models import load_model

ann = load_model('data/ann.h5')

In [18]:
y_test_pred = ann.predict(X_test_he)
y_test_pred = pd.DataFrame(y_test_pred)

y_test_pred.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,6.052655e-07,0.000018,2.562487e-08,1.082406e-05,0.000015,2.583935e-08,4.115626e-07,3.970458e-05,4.396355e-07,0.000024,2.446957e-03,6.299924e-06,1.311629e-05,2.635661e-09,0.993111,0.004200,8.767611e-05,1.751699e-05,4.169845e-08,7.889142e-06
1,4.091837e-04,0.059025,5.255918e-03,2.389544e-01,0.000416,1.836112e-04,3.103049e-03,1.025801e-02,3.156777e-02,0.268578,1.054428e-01,9.247775e-03,7.617819e-06,7.272960e-02,0.127229,0.009128,3.279475e-04,3.607989e-03,4.644892e-02,8.079321e-03
2,3.006918e-01,0.000240,4.923883e-07,9.761249e-06,0.000345,2.884817e-08,1.769841e-06,2.026321e-05,8.833738e-08,0.006712,4.064704e-04,1.894540e-04,2.530704e-02,3.203857e-04,0.548690,0.116727,3.339860e-04,6.179297e-07,1.738843e-09,3.508822e-06
3,3.798417e-07,0.000265,1.747803e-05,4.328383e-07,0.000003,1.401554e-05,4.903805e-06,5.257015e-05,3.656344e-04,0.000008,1.028277e-01,8.906941e-01,1.080864e-09,6.588465e-06,0.000005,0.000002,1.788897e-06,1.010630e-03,4.537182e-06,4.716773e-03
4,1.579379e-07,0.000048,1.188691e-10,9.056105e-09,0.000002,2.750606e-09,5.347367e-06,3.854594e-09,4.408682e-01,0.000006,1.090888e-08,4.504153e-10,5.830219e-07,2.368519e-07,0.000004,0.000001,1.831358e-10,5.526117e-09,5.590651e-01,2.269020e-10


In [65]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

seuils = np.arange(.001, .251, .001)
seuils = [round(x, 3) for x in seuils]

dico = dict(zip([i for i in range(len(seuils))], seuils))

means = []
scores = []

for seuil in seuils:
    y_test_pred_tmp = (y_test_pred.copy() > seuil) * 1 
    
    s1 = accuracy_score(test_labels, y_test_pred_tmp)
    s2 = precision_score(test_labels, y_test_pred_tmp, average = 'samples', zero_division = 1)
    s3 = recall_score(test_labels, y_test_pred_tmp, average = 'samples')
    
    # a verifier a quoi correspond sample
    
    S = [s1, s2, s3]
    
    scores.append(S)
    
    mean = np.mean(S)
    means.append(mean)
    
seuil = dico[np.argmax(means)]

print(f"seuil retenu : {seuil}\n")
print(f"accuracy score : {100 * round(scores[best][0], 3)} %")
print(f"precision score : {100 * round(scores[best][1], 3)} %")
print(f"recall score : {100 * round(scores[best][2], 3)} %")

seuil retenu : 0.137

accuracy score : 59.099999999999994 %
precision score : 81.2 %
recall score : 79.5 %


In [67]:
df_res_sup = data.copy()

X = vectorizer.fit_transform(data.post.to_list())
X = pd.DataFrame(data = X.toarray(),
                 columns = vectorizer.vocabulary)

y = ann.predict(X)
y = pd.DataFrame(y)
y = (y > seuil) * 1
y = mlb.inverse_transform(y.values)
y = [list(x) for x in y]

df_res_sup["tags_predicted_supervised"] = y
df_res_sup.head()

,post,tags,tags_predicted_supervised
Id,,,
65526399,node taking hour employee recordsgroovy xmlbe...,[javascript],[python]
65526401,python usingi based ge ge begin create called...,[python],[python]
65526407,remove ver powered whmcomplete solution whmcsi...,"[php, css]",[php]
65526419,object allocated allocate constructremoved des...,[c++],[c++]
65526447,react route change createbrowserhistory blockr...,"[javascript, reactjs]",[reactjs]


In [70]:
pickle.dump(df_res_sup, open("resultats/df_supervised.pkl", "wb"))

In [72]:
df_res_sup = pickle.load(open("resultats/df_supervised.pkl", "rb"))
df_res_sup.head()

,post,tags,tags_predicted_supervised
Id,,,
65526399,node taking hour employee recordsgroovy xmlbe...,[javascript],[python]
65526401,python usingi based ge ge begin create called...,[python],[python]
65526407,remove ver powered whmcomplete solution whmcsi...,"[php, css]",[php]
65526419,object allocated allocate constructremoved des...,[c++],[c++]
65526447,react route change createbrowserhistory blockr...,"[javascript, reactjs]",[reactjs]


In [110]:
df_tmp = df_res_sup.copy()

df_tmp.tags = df_tmp.tags.map(set)
df_tmp.tags_predicted_supervised = df_tmp.tags_predicted_supervised.map(set)

df_tmp["accuracy_score"] = (df_tmp.tags == df_tmp.tags_predicted_supervised) * 1 

def get_score(row):
    return len(row["tags"] & row["tags_predicted_supervised"]) / len(row["tags"])

df_tmp["precision_score"] = df_tmp[["tags", "tags_predicted_supervised"]].apply(get_score, axis = 1)
df_tmp.head()

,post,tags,tags_predicted_supervised,accuracy_score,precision_score
Id,,,,,
65526399,node taking hour employee recordsgroovy xmlbe...,{javascript},{python},0,0.0
65526401,python usingi based ge ge begin create called...,{python},{python},1,1.0
65526407,remove ver powered whmcomplete solution whmcsi...,"{css, php}",{php},0,0.5
65526419,object allocated allocate constructremoved des...,{c++},{c++},1,1.0
65526447,react route change createbrowserhistory blockr...,"{reactjs, javascript}",{reactjs},0,0.5


In [111]:
print(f"precision score : {round(df_tmp.precision_score.mean(), 3)}")
print(f"accuracy score : {round(df_tmp.accuracy_score.mean(), 3)}")

precision score : 0.831
accuracy score : 0.627


# Partie non supervisée

In [16]:
from gensim.models import TfidfModel
import gensim.corpora as corpora
from gensim import models
from gensim.utils import simple_preprocess
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import matplotlib.pyplot as plt

lda_model = pickle.load(open("data/lda_model.pkl", "rb"))
tfidf_corpus = pickle.load(open("data/tfidf_lda.pkl", "rb"))

df = data.copy()
df.head()

,post,tags
Id,,
65526399,node taking hour employee recordsgroovy xmlbe...,[javascript]
65526401,python usingi based ge ge begin create called...,[python]
65526407,remove ver powered whmcomplete solution whmcsi...,"[php, css]"
65526419,object allocated allocate constructremoved des...,[c++]
65526447,react route change createbrowserhistory blockr...,"[javascript, reactjs]"


In [17]:
texts = df.post.str.split().to_list()

id2word = corpora.Dictionary()
vocabulary = pickle.load(open("data/list_vocab.pkl", "rb"))
id2word.add_documents([[x] for x in vocabulary])

# bow_corpus = [id2word.doc2bow(text) for text in texts]
# tfidf = TfidfModel(bow_corpus)
# tfidf_corpus = [tfidf[text] for text in bow_corpus]

In [16]:
model = LdaMulticore(corpus = tfidf_corpus,
                    id2word = id2word,
                    num_topics = 20, # nombre de tags retenus
                    random_state = 42,
                    passes = 10,
                    workers = 7,
                    minimum_probability = 0)

In [13]:
# 41 569 560 Mo de RAM 

In [12]:
model.log_perplexity(tfidf_corpus)

-6.650407164120078

In [14]:
coherencemodel = CoherenceModel(model = model, 
                                texts = texts,
                                dictionary = id2word, 
                                coherence = 'c_v')

coherencemodel.get_coherence()

0.6361065489253304

In [8]:
from pprint import pprint

model_topics = lda_model.show_topics(formatted=False)
pprint(lda_model.print_topics(num_words=30))

[(0,
  '0.034*"convert" + 0.024*"read" + 0.020*"string" + 0.019*"key" + '
  '0.018*"dict" + 0.016*"exple" + 0.014*"json" + 0.013*"format" + 0.013*"ve" + '
  '0.012*"variable" + 0.012*"tried" + 0.012*"object" + 0.012*"don" + '
  '0.012*"question" + 0.012*"trying" + 0.012*"gl" + 0.011*"extension" + '
  '0.011*"help" + 0.011*"thanks" + 0.011*"image" + 0.010*"able" + 0.010*"time" '
  '+ 0.010*"create" + 0.010*"result" + 0.010*"multiple" + 0.010*"using" + '
  '0.010*"buffer" + 0.009*"print" + 0.009*"following" + 0.009*"text"'),
 (1,
  '0.059*"andro" + 0.041*"val" + 0.036*"fun" + 0.035*"activity" + '
  '0.033*"intent" + 0.025*"layout" + 0.024*"overre" + 0.019*"dp" + '
  '0.019*"permission" + 0.018*"fragment" + 0.016*"view" + 0.015*"recyclerview" '
  '+ 0.014*"savedinstancestate" + 0.013*"context" + 0.013*"java" + '
  '0.013*"mainactivity" + 0.012*"tag" + 0.011*"wrap" + 0.011*"androx" + '
  '0.010*"item" + 0.010*"parent" + 0.009*"content" + 0.009*"xmlns" + '
  '0.008*"com" + 0.008*"public" + 

In [5]:
def format_topics_sentences(ldamodel, corpus):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model,
                                                  corpus=tfidf_corpus)
df_topic_sents_keywords

df_topic_sents_keywords.index = df.index
df_topic_sents_keywords.head()

,Dominant_Topic,Perc_Contribution,Topic_Keywords
Id,,,
65526399,6.0,0.6443,"py, python, install, site, package, usr, run, ..."
65526401,2.0,0.7756,"self, def, view, print, func, var, nil, swiftu..."
65526407,16.0,0.5480,"request, www, url, php, socket, server, page, ..."
65526419,11.0,0.4584,"std, foo, int, template, operator, vector, cpp..."
65526447,9.0,0.8365,"react, component, const, div, prop, props, use..."


In [7]:
df_tmp = df_topic_sents_keywords.copy()
df_tmp.sort_values(by = ["Perc_Contribution"], ascending = False, inplace = True)
df_tmp.drop_duplicates(subset = ["Dominant_Topic"], inplace = True)
df_tmp = df_tmp.join(df[["tags"]])

df_tmp[["Dominant_Topic", "Topic_Keywords", "tags"]]

,Dominant_Topic,Topic_Keywords,tags
Id,,,
61372980,14.0,"child, context, wget, icon, dart, firestore, b...",[flutter]
60090428,17.0,"node, js, module, react, npm, modules, angular...",[reactjs]
66277275,16.0,"request, www, url, php, socket, server, page, ...",[reactjs]
67265453,3.0,"public, org, java, microsoft, string, method, ...",[c#]
59567602,19.0,"aws, ec, access, role, bucket, cluster, resour...","[node.js, amazon-web-services]"
61418553,7.0,"await, const, log, async, err, message, respon...",[node.js]
62308701,8.0,"google, studio, andro, flutter, play, device, ...","[android, flutter]"
63313952,1.0,"andro, val, fun, activity, intent, layout, ove...",[android]
60908554,13.0,"div, px, span, style, td, li, img, button, col...",[javascript]


In [19]:
df_tmp = df.copy()
df_tmp.post = df_tmp.post.str.split()

def predict_unsupervised_tags(text, lda_model, id2w):
    """
    Predict tags of a preprocessed text
    
    Args:
        text(list): preprocessed text
        
    Returns:
        relevant_tags(list): list of tags
    """
    corpus_new = id2w.doc2bow(text)
    topics = lda_model.get_document_topics(corpus_new)
    
    #find most relevant topic according to probability
    relevant_topic = topics[0][0]
    relevant_topic_prob = topics[0][1]
    
    for i in range(len(topics)):
        if topics[i][1] > relevant_topic_prob:
            relevant_topic = topics[i][0]
            relevant_topic_prob = topics[i][1]
            
    #retrieve associated to topic tags present in submited text
    potential_tags = lda_model.get_topic_terms(topicid=relevant_topic, topn=20)
    
    relevant_tags = [id2w[tag[0]] for tag in potential_tags if id2w[tag[0]] in text]
    
    return relevant_tags

for i in range(30):
    print(f"Liste de tags à prédire : {df.sample(30, random_state = 0).tags.iloc[i]}\n")
    print(f"Liste des tags prédits par le modèle non supervisé: {predict_unsupervised_tags(df_tmp.sample(30, random_state = 0).post.iloc[i],lda_model,id2word)}\n\n")

Liste de tags à prédire : ['javascript', 'html', 'css']

Liste des tags prédits par le modèle non supervisé: ['color', 'black', 'label']


Liste de tags à prédire : ['angular']

Liste des tags prédits par le modèle non supervisé: ['node', 'js', 'module', 'npm', 'angular', 'test', 'run', 'ts', 'script', 'cli', 'err']


Liste de tags à prédire : ['javascript', 'angular']

Liste des tags prédits par le modèle non supervisé: ['span', 'style', 'button', 'content', 'body', 'btn', 'link', 'wth', 'html', 'item', 'flex']


Liste de tags à prédire : ['python', 'python-3.x']

Liste des tags prédits par le modèle non supervisé: ['color', 'plt', 'label', 'gr']


Liste de tags à prédire : ['android', 'flutter']

Liste des tags prédits par le modèle non supervisé: ['andro', 'flutter', 'run', 'plugin', 'project', 'build', 'version', 'firebase', 'failed']


Liste de tags à prédire : ['python', 'python-3.x']

Liste des tags prédits par le modèle non supervisé: ['py', 'python', 'install', 'pip', 'line', 

In [20]:
df_res_unsup = data.copy()
df_res_unsup["tags_predicted_unsupervised"] = df_res_unsup.post.map(lambda x: predict_unsupervised_tags(x.split(), 
                                                                                            lda_model,
                                                                                            id2word))
pickle.dump(df_res_unsup, open("resultats/df_unsupervised.pkl", "wb"))
df_res_unsup.head()

,post,tags,tags_predicted_unsupervised
Id,,,
65526399,node taking hour employee recordsgroovy xmlbe...,[javascript],"[home, test]"
65526401,python usingi based ge ge begin create called...,[python],"[self, def, print, init, else]"
65526407,remove ver powered whmcomplete solution whmcsi...,"[php, css]","[don, question, help]"
65526419,object allocated allocate constructremoved des...,[c++],[std]
65526447,react route change createbrowserhistory blockr...,"[javascript, reactjs]","[react, const, prop, useeffect, route, button,..."


In [73]:
df_res_unsup = pickle.load(open("resultats/df_unsupervised.pkl", "rb"))
df_res_unsup.head()

,post,tags,tags_predicted_unsupervised
Id,,,
65526399,node taking hour employee recordsgroovy xmlbe...,[javascript],"[home, test]"
65526401,python usingi based ge ge begin create called...,[python],"[self, def, print, init, else]"
65526407,remove ver powered whmcomplete solution whmcsi...,"[php, css]","[don, question, help]"
65526419,object allocated allocate constructremoved des...,[c++],[std]
65526447,react route change createbrowserhistory blockr...,"[javascript, reactjs]","[react, const, prop, useeffect, route, button,..."


In [107]:
mlb_unsup = MultiLabelBinarizer()
mlb_unsup.fit(df_res_unsup.tags_predicted_unsupervised)

tags_detectes = set(mlb.classes_) & set(mlb_unsup.classes_)

print(f"Nombre de tags à prédire : {len(mlb.classes_)}\n")
print(f"Tags détectés par notre modèle non supervisé : {tags_detectes}\n")
print(f"Nombre de tags manquants : {len(mlb.classes_) -  len(tags_detectes)}\n")
print(f"Nombre de tags en trop : {len(MLB.classes_) - len(tags_detectes)}")

Nombre de tags à prédire : 20

Tags détectés par notre modèle non supervisé : {'angular', 'flutter', 'python', 'ios', 'pandas', 'java', 'html', 'php'}

Nombre de tags manquants : 12

Nombre de tags en trop : 317


In [104]:
df_tmp = df_res_unsup.copy()

df_tmp.tags = df_tmp.tags.map(set)
df_tmp.tags_predicted_unsupervised = df_tmp.tags_predicted_unsupervised.map(set)

df_tmp["accuracy_score"] = (df_tmp.tags == df_tmp.tags_predicted_unsupervised) * 1 

def get_score(row):
    return len(row["tags"] & row["tags_predicted_unsupervised"]) / len(row["tags"])

df_tmp["precision_score"] = df_tmp[["tags", "tags_predicted_unsupervised"]].apply(get_score, axis = 1)
df_tmp.head()

,post,tags,tags_predicted_unsupervised,accuracy_score,precision_score
Id,,,,,
65526399,node taking hour employee recordsgroovy xmlbe...,{javascript},"{test, home}",0,0.0
65526401,python usingi based ge ge begin create called...,{python},"{init, print, else, self, def}",0,0.0
65526407,remove ver powered whmcomplete solution whmcsi...,"{css, php}","{don, help, question}",0,0.0
65526419,object allocated allocate constructremoved des...,{c++},{std},0,0.0
65526447,react route change createbrowserhistory blockr...,"{reactjs, javascript}","{const, useeffect, button, log, route, page, p...",0,0.0


In [106]:
print(f"precision score : {round(df_tmp.precision_score.mean(), 3)}")
print(f"accuracy score : {round(df_tmp.accuracy_score.mean(), 3)}")

precision score : 0.092
accuracy score : 0.002
